In [2]:
import os
import glob
import subprocess
import shutil
import logging
from datetime import datetime
import psycopg2

In [12]:
# configuration
INPUT_DIR = "/mnt/c/Users/bzwil/OneDrive/Desktop/KaliningradProject/kaliningrad-geoint-change-detection/files/data/prepared_data"
DB_NAME = "kaliningraddata"
DB_USER = "postgres"
DB_PASSWORD="200394121"
DB_HOST = "localhost"
DB_PORT = "5432"
TABLE = "public.dem_clipped"
SRS = "4326"
TILE_SIZE="100x100"
LOG_FILE="/mnt/c/Users/bzwil/OneDrive/Desktop/KaliningradProject/kaliningrad-geoint-change-detection/files/transfer.log"
TIMEOUT_SECONDS=600

In [ ]:
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# ensuring the correct file path
def find_raster2pgsql():
    possible_path = [
        "/usr/bin/raster2pgsql"
    ]
    
    for path in possible_path: 
        if os.path.isfile(path) and os.access(path, os.X_OK):
            return path  
    return shutil.which("raster2pgsql")

def main():
    logging.info(f"pipeline start: {datetime.now()}")
    
    # checking to see if input dir exists
    if not os.path.isdir(INPUT_DIR):
        logging.error("input dir doesn't exist")
        raise FileNotFoundError("input dir doesn't exist")
    
    # checking is raster path is valid
    raster2pgsql_path = find_raster2pgsql()
    if not raster2pgsql_path:
        logging.error("raster2pgsql not found. Install postgresql-16-postgis-3.")
        raise FileNotFoundError("raster2pgsql not found. Install postgresql-16-postgis-3.")
    
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
        )
    except Exception as e:
        logging.error(f"Database connection failed: {str(e)}")
        raise
    
    # checking dem file existence 
    dem_files = glob.glob(os.path.join(INPUT_DIR, "dem_clipped.tif"))
    if not dem_files:
        logging.error(f"No .tif files found in {INPUT_DIR}")
        raise FileNotFoundError(f"No .tif files found in {INPUT_DIR}") 
    
    for dem_file in dem_files:
        filename = os.path.basename(dem_file)
        logging.info(f"processing {filename}")
        
        # raster2pgsql command to execute file transfer
        raster2pgsql_cmd = [
             raster2pgsql_path,
            "-s", SRS,
            "-I", "-C", "-M",
            "-F",
            "-t", TILE_SIZE,
            dem_file,
            TABLE
        ]
        
        # sequential psql command
        psql_cmd = [
            "psql",
            "-U", DB_USER,
            "-h", DB_HOST,
            "-p", DB_PORT,
            "-d", DB_NAME
        ]

        try:
            raster2pgsql_proc = subprocess.Popen(
                raster2pgsql_cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            psql_proc = subprocess.Popen(
                psql_cmd,
                stdin=raster2pgsql_proc.stdout,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            raster2pgsql_proc.stdout.close() 
            stdout, stderr = psql_proc.communicate(timeout=TIMEOUT_SECONDS)

            if psql_proc.returncode == 0:
                logging.info(f"Successfully loaded {filename}")
            else:
                logging.error(f"Error loading {filename}: {stderr.decode()}")
                raise subprocess.CalledProcessError(psql_proc.returncode, psql_cmd, stderr)

        except Exception as e:
            logging.error(f"Error loading {filename}: {str(e)}")
            raise

    logging.info(f"Pipeline completed at {datetime.now()}")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logging.error(f"Pipeline failed: {str(e)}")
        exit(1)